<월 단위(결과 보고 어쩌면 주단위)>

- 과재고 상품 TOP20
    - 제품명/재고량/지난4주 주평균 판매량/재고회전주수(정렬기준)
- 부족재고 상품 TOP20
    - 제품명/마지막재고일/재고있을 때 평균 판매량(정렬기준)

In [1]:
from db_import import *
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Data Import
end_date = datetime.now().strftime('%Y-%m-%d')
query_obj = Queries('2022-01-01', end_date)
db_obj = DBImport(db_type='cscart')

analytics = db_obj.data_import(query_obj.analytics_query)
brand = db_obj.data_import(query_obj.brand_query)
inventory = db_obj.data_import(query_obj.inventory_query)
# category = db_obj.data_import(query_obj.category_query)
# mainexposure = db_obj.data_import(query_obj.mainexposure_query)

In [3]:
# basic preprocessing
analytics['purchased_ymd'] = pd.to_datetime(analytics.purchased_at).dt.normalize() # 시간 제외한 날짜만
inventory['sys_time'] = pd.to_datetime(inventory['sys_time']).dt.normalize() # 시간 제외한 날짜만
brand['brand'] = brand['brand'].apply(lambda x: x.strip())

In [4]:
# product-barcode matching table
prod_bar_match = analytics[['product_id', 'barcode']].drop_duplicates()

# barcode-product_name matching table (추후 바코드 없는 상품들은 바코드 기준 조인이 안되기 때문에 상품명, 옵션명 테이블 따로 생성)
bar_prod_name = analytics[['product_id', 'product_name_kor']].drop_duplicates()
bar_prod_name = bar_prod_name.groupby('product_id')['product_name_kor'].sum().reset_index() # 한 상품명 내에서 여러 옵션이 담겨있기도 함. 모든 옵션을 보여주기 위해 합치기

# barcode-variant_name matching table (추후 바코드 없는 상품들은 바코드 기준 조인이 안되기 때문에 상품명, 옵션명 테이블 따로 생성)
bar_var_name = analytics[['barcode', 'variant_1_name_kor']].drop_duplicates()
bar_var_name = bar_var_name.groupby('barcode')['variant_1_name_kor'].sum().reset_index() # 한 바코드 내에서 여러 옵션이 담겨있기도 함. 모든 옵션을 보여주기 위해 합치기

In [5]:
# 추후 컬럼명 한국어로 변경 시 공통 활용되는 dictionary
col_name_dict = {
    'barcode': '바코드',
    'amount': '현재고',
    'sys_time': '재고일자',
    'product_qty': '주평균판매수량',
    'wio': '재고회전주수',
    'product_id': '상품번호',
    'brand': '브랜드',
    'product_name_kor': '상품명',
    'variant_1_name_kor': '옵션명',
    'last_stock_ymd': '최근재고보유일자',
    'oos_days': '결품경과일수'
}

### 과다재고 상품
- 제품명/재고량/지난4주 주평균 판매량/재고회전주수(정렬기준)

In [6]:
# 현재고
now_date = inventory['sys_time'].max()
now_inventory = inventory[inventory['sys_time']==now_date]

last_month_sales = analytics[(analytics['purchased_ymd']<=now_date)&(analytics['purchased_ymd']>(now_date-timedelta(days=28)))]
last_month_sales = last_month_sales.groupby(['barcode', 'purchased_ymd'])['product_qty'].sum().reset_index()
last_month_sales = last_month_sales.rename(columns={'purchased_ymd': 'sys_time'})
last_month_inventory = inventory[(inventory['sys_time']<=now_date)&(inventory['sys_time']>(now_date-timedelta(days=28)))] # 동일한 기간의 재고와 merge. 재고가 있는 기간의 판매량만 보기 위함(재고가 0인 경우 제외, 재고가 있는데 판매량이 집계되지 않는 경우 0으로 채우기 위함)
last_month_sales = pd.merge(last_month_inventory, last_month_sales, on=['barcode', 'sys_time'], how='outer')
last_month_sales = last_month_sales[~((last_month_sales['barcode'].isin(['', '0', 0]))|(last_month_sales['barcode'].isnull()))]
last_month_sales = last_month_sales.fillna(0)

# 지난4주 주평균 판매량
avg_week_sales = last_month_sales[last_month_sales['amount']>0].groupby('barcode')['product_qty'].mean().reset_index() # 재고가 있을 때 일평균 판매량
avg_week_sales['product_qty'] = np.ceil(avg_week_sales['product_qty'] * 7 ) # 일평균 판매량에 7곱하여 주평균 판매량 구한 후 ceiling

over_stock = pd.merge(now_inventory, avg_week_sales, on='barcode', how='left')
over_stock['wio'] = over_stock['amount'] / (over_stock['product_qty'].apply(lambda x: 1 if x==0 else x)) # week of inventory outstanding (재고자산 회전주수) # 평균판매량이 0일 경우, 1로 보정해줌. 비교기간동안 한 번도 팔리지 않았다는 의미인데, 그냥 둘 경우 inf가 나와서 비교가 어려움. 1로 넣고 재고 많은 순서대로 뜰 수 있도록. 그 외 추가로 주평균판매량인 product_qty는 그대로 살려둠으로써, 한번도 팔리지 않았는지, 주평균판매량이 1인지 구분은 할 수 있도록 작업
over_stock = over_stock.sort_values(by='wio', ascending=False)

# 후처리
over_stock = pd.merge(over_stock, prod_bar_match, on='barcode', how='left')
over_stock = pd.merge(over_stock, brand, on='product_id', how='left')
over_stock = pd.merge(over_stock, bar_prod_name, on='product_id', how='left') # barcode 없는 상품들로 인해 상품명과 옵션명 별도로 조인
over_stock = pd.merge(over_stock, bar_var_name, on='barcode', how='left')
over_stock = over_stock.rename(columns=col_name_dict)

In [7]:
over_stock

,바코드,현재고,재고일자,주평균판매수량,재고회전주수,상품번호,브랜드,상품명,옵션명
0,0736097000011,424,2022-10-03,1.00,424.00,3035.00,Valeo,코튼 리프팅 스트랩,한쌍
1,0651116001315,2305,2022-10-03,8.00,288.12,5283.00,Cardillo,카딜로 나일론 스트링 백,Black
2,0752830827400,2555,2022-10-03,9.00,283.89,3490.00,R3,R3 아사이 1000mg 120식물성캡슐,120 식물성캡슐
3,0752830827301,3057,2022-10-03,11.00,277.91,3498.00,R3,R3 비타민 D3 5000 IU 120소프트젤,120 소프트젤
4,0651595001516,1576,2022-10-03,6.00,262.67,3121.00,Monsterzym,몬스터짐 물병 2.2리터,2.2 리터
...,...,...,...,...,...,...,...,...,...
2590,08222209,0,2022-10-03,NaN,NaN,NaN,NaN,NaN,NaN
2591,08222210,0,2022-10-03,NaN,NaN,NaN,NaN,NaN,NaN
2592,08222211,0,2022-10-03,NaN,NaN,NaN,NaN,NaN,NaN
2593,08222212,0,2022-10-03,NaN,NaN,NaN,NaN,NaN,NaN


### 과소재고 상품
- 제품명/마지막재고일/재고있을 때 평균 판매량(정렬기준)

In [8]:
# 추후 부족재고 정보도 확인하기 위해 재고일자와 판매일자 merge
last_6month_sales = analytics[(analytics['purchased_ymd']<=now_date)&(analytics['purchased_ymd']>(now_date-timedelta(days=180)))]
last_6month_sales = last_6month_sales.groupby(['barcode', 'purchased_ymd'])['product_qty'].sum().reset_index()
last_6month_sales = last_6month_sales.rename(columns={'purchased_ymd': 'sys_time'})
last_6month_inventory = inventory[(inventory['sys_time']<=now_date)&(inventory['sys_time']>(now_date-timedelta(days=180)))] # 동일한 기간의 재고와 merge (재고가 0인 경우 제외, 재고가 있는데 판매량이 집계되지 않는 경우 0으로 채우기 위함)
last_6month_sales = pd.merge(last_6month_inventory, last_6month_sales, on=['barcode', 'sys_time'], how='outer')
last_6month_sales = last_6month_sales[~((last_6month_sales['barcode'].isin(['', '0', 0]))|(last_6month_sales['barcode'].isnull()))]
last_6month_sales = last_6month_sales.fillna(0)

# 주평균 판매량
avg_week_sales = last_6month_sales[last_6month_sales['amount']>0].groupby('barcode')['product_qty'].mean().reset_index() # 재고가 있을 때 일평균 판매량
avg_week_sales['product_qty'] = np.ceil(avg_week_sales['product_qty'] * 7 ) # 일평균 판매량에 7곱하여 주평균 판매량 구한 후 ceiling

# 마지막으로 재고 있었던 날
last_stock = inventory[inventory['amount']>0].groupby('barcode')['sys_time'].max().reset_index()
last_stock = last_stock.rename(columns={'sys_time': 'last_stock_ymd'})
low_stock = pd.merge(last_stock, avg_week_sales, on='barcode', how='left')
low_stock['product_qty'] = low_stock['product_qty'].fillna(0)
low_stock['oos_days'] = low_stock['last_stock_ymd'].apply(lambda x: (now_date - x).days)
low_stock = low_stock[low_stock['last_stock_ymd']!=now_date]
low_stock = low_stock.sort_values(by='product_qty', ascending=False)

# 후처리
low_stock = pd.merge(low_stock, prod_bar_match, on='barcode', how='left')
low_stock = pd.merge(low_stock, brand, on='product_id', how='left')
low_stock = pd.merge(low_stock, bar_prod_name, on='product_id', how='left') # barcode 없는 상품들로 인해 상품명과 옵션명 별도로 조인
low_stock = pd.merge(low_stock, bar_var_name, on='barcode', how='left')
low_stock = low_stock.rename(columns=col_name_dict)

In [9]:
low_stock

,바코드,최근재고보유일자,주평균판매수량,결품경과일수,상품번호,브랜드,상품명,옵션명
0,0672898123422,2022-08-25,332.00,39,5576.00,S.A.N,SAN 퍼포먼스 크레아틴 120서빙,600 g
1,0834266001590,2022-09-20,208.00,13,1574.00,BSN,"엔오 익스플로드 60서빙, 미국버전노익스플로드 60서빙, 미국버전",포도
2,0651116000127,2022-09-25,170.00,8,314.00,Cardillo,카딜로 디럭스 310DL 벨트,스몰
3,0842595107418,2022-05-06,144.00,150,4109.00,Cellucor,C4 프리워크아웃 60서빙,Pineapple파인애플
4,0651116000615,2022-07-13,105.00,82,4180.00,Cardillo,카딜로 여성 웨이트 벨트 핑크,X-Small
...,...,...,...,...,...,...,...,...
1846,0651561001045,2021-05-30,0.00,491,NaN,NaN,NaN,NaN
1847,0651561001038,2021-06-29,0.00,461,NaN,NaN,NaN,NaN
1848,0651561001021,2021-06-25,0.00,465,NaN,NaN,NaN,NaN
1849,0651370000017,2021-08-04,0.00,425,NaN,NaN,NaN,NaN


22.10.3
- 재고의 유무에 따라 판매량 집계가 왜곡될 수 있음
    - 재고가 없을 때 판매량을 0으로 집계한다면 낮아질 수 있음. 따라서, 재고가 있을 때의 판매량으로만 집계
    - 재고가 있을 때의 판매량(0판매는 0으로 복구)을 집게하여 일단위 판매량을 구한 후, 7을 곱하여 주단위 판매량을 연산. 그 후 ceiling을 통해 0~1사이의 판매량을 보이는 경우에는 1로 보정
- 과다재고의 경우 최근 판매 추이도 중요하기 때문에 주평균판매량을 연산할 때 최근 4주를 대상으로 연산. 하지만, 과소재고의 경우 판매량은 많지만 입고가 되지 않는 상품들을 찾는 것이 문제이기 때문에, 조금 더 길게, 최근 6개월의 데이터를 대상으로 연산.
- 단, 재고의 경우 입고 및 판매가 다 이루어진 결과이기 때문에, 당일 입고된 수량이 전량 판매되었을 경우, 재고가 0으로 나올 수도 있음